In [7]:
#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

import spacy
from spacy.matcher import Matcher

from tqdm import tqdm

nlp = spacy.load("es_core_news_md")

In [8]:

df = pd.read_json("data/tweets_0.json", orient = 'records', lines = True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 36 columns):
_id                          9999 non-null object
contributors                 0 non-null float64
coordinates                  361 non-null object
created_at                   9999 non-null datetime64[ns]
display_text_range           5573 non-null object
entities                     9999 non-null object
extended_entities            1325 non-null object
extended_tweet               1873 non-null object
favorite_count               9999 non-null int64
favorited                    9999 non-null bool
filter_level                 9999 non-null object
geo                          361 non-null object
id                           9999 non-null int64
id_str                       9999 non-null int64
in_reply_to_screen_name      4791 non-null object
in_reply_to_status_id        4508 non-null float64
in_reply_to_status_id_str    4508 non-null float64
in_reply_to_user_id          4791 non-null

In [12]:
#Primero convertimos todo el teto a lower case para evitar que cada palabra se cuente doble
df['text']=df.text.str.lower()

In [13]:
df['text']

0       @beatrizgmuller @lopezobrador_ @hlgatell @john...
1                                             pue' ser...
2       @fermg33_ no quiero ser mamona, pero le falta ...
3       @webcamsdemexico @pueblaesmexico un cocodrilo ...
4               #lopezlargateya #lopezfracasopresidencial
5                                  ⚠️👇🏻⚠️👇🏻⚠️👇🏻⚠️👇🏻⚠️👇🏻⚠️
6            @atalia95 luego esto https://t.co/hlyjcpumjf
7                         qué pinches ganas de cosa seria
8                                me recordó a alguien 🙃🙄🤩
9              @manuevas acá en mex les decimos cachondos
10                                   pero qué necesidad 😔
11      todos necesitamos ese momento de oscuridad par...
12      @hectorr18264358 @vanhot7 ase. es. te aborda q...
13      - incendia la casa y culpa a winston churchill...
14      @sinlineamx de q se preocupan\nson perros ladr...
15                 @ameliayanez_ jajaja ya tiene rato eso
16      ya le quiero contar a nuestros hijos que su ma...
17      @luigu

In [14]:
#Procederemos a lematizar los tweets
salida=[]
for i in df.text:
    doc=nlp(i)
    for token in doc:
        salida.append(token.lemma_)

In [16]:
len(set(salida))

20728

In [17]:
#Palabras frecuentes
import collections
import operator
unicas=collections.Counter(salida)
unicas_sort=sorted(unicas.items(), key=operator.itemgetter(1), reverse = True)

In [18]:
unicas_sort[:200]

[('de', 3382),
 (',', 3334),
 ('lo', 3013),
 ('que', 3003),
 ('.', 2764),
 ('el', 2488),
 ('#', 2082),
 ('ser', 2072),
 ('a', 2027),
 ('!', 2025),
 ('no', 2000),
 ('en', 1949),
 ('y', 1917),
 ('…', 1879),
 ('uno', 1354),
 ('\n', 1216),
 ('?', 1073),
 ('me', 1042),
 ('se', 948),
 ('estar', 865),
 ('por', 786),
 ('mi', 740),
 ('con', 704),
 ('si', 694),
 ('este', 692),
 ('ya', 690),
 ('...', 632),
 ('tener', 612),
 ('hacer', 609),
 ('parir', 607),
 ('haber', 560),
 ('todo', 554),
 ('los', 502),
 ('te', 501),
 ('_', 496),
 ('ir', 477),
 (' ', 454),
 ('su', 450),
 ('comer', 444),
 ('ver', 439),
 ('️', 437),
 ('del', 435),
 ('\n\n', 433),
 ('pero', 431),
 ('casar', 425),
 ('decir', 416),
 ('al', 400),
 ('😂', 394),
 ('ese', 378),
 ('más', 365),
 ('yo', 363),
 ('querer', 346),
 ('tu', 340),
 ('@hlgatell', 333),
 ('"', 329),
 ('poder', 324),
 ('🤣', 301),
 (':', 297),
 ('le', 290),
 ('¿', 278),
 ('@lopezobrador', 274),
 ('día', 255),
 ('o', 251),
 ('qué', 242),
 ('salir', 236),
 ('asir', 224),


In [19]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [21]:
from nltk.corpus import stopwords
stopwords=[word for word in stopwords.words('spanish')]

vectorizer = TfidfVectorizer(stopwords)
X=vectorizer.fit_transform(df.text)
true_k = 4
model= KMeans(n_clusters=true_k, init='K-means++', max_iter=10000, n_init=1)
model.fit(X)

ValueError: the init parameter for the k-means should be 'k-means++' or 'random' or an ndarray, 'K-means++' (type '<class 'str'>') was passed.